In [1]:
import requests

url = "http://localhost:5000/chat"
payload = {"text": "I feel down and exhausted lately."}

response = requests.post(url, json=payload)
print(response.json())

{'mood': 'Angry', 'recommendations': ['Boston - Hitch a Ride', 'Red Hot Chili Peppers - Nobody Weird Like Me - Remastered', 'Arctic Monkeys - The View From The Afternoon', "Buffalo Springfield - For What It's Worth", 'Velvet Two Stripes - Drinks', 'The Stone Roses - Driving South', 'Better Than Ezra - Good', 'Silversun Pickups - Lazy Eye', 'U2 - The Blackout', 'The Clash - The Magnificent Seven - Remastered', 'The Last Shadow Puppets - Bad Habits', 'Sly & The Family Stone - I Want To Take You Higher - Live at The Woodstock Music & Art Fair, August 17, 1969', 'Berry Sakharof - ◊†◊§◊™◊ú◊ô ◊î◊ì◊í', 'Soen - Lotus', 'Liam Gallagher - Once', 'El Goodo - Feel so Fine', 'Why Bonnie - Athlete', 'MOD SUN - Stay Away (feat. Machine Gun Kelly & Goody Grace)', 'The Black Crowes - Sometimes Salvation', 'Radiohead - Burn the Witch', 'Arctic Monkeys - R U Mine?', 'The Stranglers - Golden Brown', 'Joe Bonamassa - Cradle Rock', "Primal Scream - Movin' on Up", 'Sharon Van Etten - Comeback Kid', 'beabadoo

In [2]:
!fluidsynth --version

'fluidsynth' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import torch
import torch.nn.functional as F
import librosa
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
import argparse

# -------------------------------
# CONFIGURATION
# -------------------------------

# Replace with your actual model directory or HF hub ID
MODEL_PATH = "models/genre_classification/model.safetensors"  # e.g., "./genre_model" or "username/model-id"

# Set computation device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and feature extractor
extractor = AutoFeatureExtractor.from_pretrained(MODEL_PATH)
model = AutoModelForAudioClassification.from_pretrained(MODEL_PATH).to(device)
model.eval()

# -------------------------------
# AUDIO PREPROCESSING FUNCTION
# -------------------------------

def preprocess_audio(audio_path, sampling_rate=16000):
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    return {"array": audio, "sampling_rate": sr}

# -------------------------------
# PREDICTION FUNCTION
# -------------------------------

def predict_genre(audio_path):
    audio_input = preprocess_audio(audio_path)
    features = extractor(audio_input["array"], sampling_rate=audio_input["sampling_rate"], return_tensors="pt")

    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in features.items()}
        logits = model(**inputs).logits
        predicted_class_id = torch.argmax(logits, dim=1).item()
        probabilities = F.softmax(logits, dim=1).cpu().numpy().squeeze()

    label = model.config.id2label[predicted_class_id]
    return label, probabilities

# -------------------------------
# CLI INTERFACE
# -------------------------------

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Genre classification from audio file")
    parser.add_argument("audio_path", type=str, help="Path to the .wav audio file")

    args = parser.parse_args()
    genre, probs = predict_genre(args.audio_path)

    print(f"\n🎶 Predicted Genre: {genre}")
    print("🎛️  Class Probabilities:")
    for i, (label_id, label_name) in enumerate(model.config.id2label.items()):
        print(f"  {label_name}: {probs[i]:.4f}")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 0: invalid continuation byte

In [5]:
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor

# Path to your extracted folder (where model.safetensors and config.json exist)
MODEL_PATH = "models/genre_classification"
SAVE_PATH = "models/genre_model_clean"

# Load and re-save model and extractor
model = AutoModelForAudioClassification.from_pretrained(MODEL_PATH, trust_remote_code=True)
extractor = AutoFeatureExtractor.from_pretrained(MODEL_PATH)

model.save_pretrained(SAVE_PATH, safe_serialization=True)
extractor.save_pretrained(SAVE_PATH)

print(f"✅ Model and extractor saved to: {SAVE_PATH}")


W0501 10:38:16.514000 38304 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


✅ Model and extractor saved to: models/genre_model_clean
